In [31]:
''' Part 1) 다이얼톤 파일 들어보기 '''
from IPython.display import Audio

dialtone_file = 'dialtone.wav'
output_file = 'output.wav'

# Audio(dialtone_file, autoplay=True)

In [ ]:
''' Part 2) 다이얼톤 파일 읽어오기 '''
from scipy.io import wavfile
import numpy as np

[sample_rate, audio_data] = wavfile.read(dialtone_file)  # sample_rate: 샘플링 레이트, audio_data: 오디오 데이터 배열

sample_count = len(audio_data)  # sample_count: 샘플 수
time_stamps = np.linspace(0, (sample_count - 1) / sample_rate, sample_count)  # time_stamps: 타임스탬프

if audio_data.dtype == 'int16':
    print("int16 형식입니다.")
    normalized_data = audio_data / 32768
elif audio_data.dtype == 'uint8':
    print("uint8 형식입니다.")
    normalized_data = audio_data / 255
else:
    print('지원하지 않는 형식입니다.')

print(['audio_data: ', audio_data])
print(sample_rate)
print(normalized_data)
print(time_stamps)

In [ ]:
''' Part 3) 시간축에서 신호의 파형을 그래프로 그리기 '''
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(18, 6), dpi=80)

plt.stem(time_stamps, normalized_data, markerfmt=" ")
plt.xlabel("Time (s)")
plt.ylabel("Normalized Amplitude")
plt.title("Dial Tone on Time Domain")
plt.grid(True)
plt.axis([0, (sample_count - 1) / sample_rate, min(normalized_data), 1])

In [ ]:
''' Part 4) 푸리에 변환 후 주파수 축에서의 그래프 '''
fft_result = np.fft.fft(normalized_data, sample_count) / sample_count
frequencies = np.linspace(0, sample_rate, sample_count)

figure(figsize=(18, 6), dpi=80)

plt.stem(frequencies, np.abs(fft_result), 'r', markerfmt=" ")
plt.grid(True)
plt.axis([0, sample_rate, 0, max(abs(fft_result))])

In [ ]:
''' Part 5) 유효 데이터에 대한 주파수 축에서 그래프 '''
half_samples = int(sample_count / 2)

filtered_fft = fft_result[:half_samples + 1]
filtered_fft[1:half_samples] *= 2
amplitude_spectrum = filtered_fft

half_frequencies = np.linspace(0, sample_rate / 2, half_samples + 1)

print(len(half_frequencies))
print(len(amplitude_spectrum))
print(half_frequencies)

figure(figsize=(18, 6), dpi=80)

plt.stem(half_frequencies, np.abs(amplitude_spectrum), 'r', markerfmt=" ")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.title("Dial Tone on Frequency Domain")
plt.grid(True)
plt.axis([0, sample_rate / 2, 0, max(abs(amplitude_spectrum))])

In [ ]:
''' Part 6) 주파수 대역 설정 '''
LOW_FREQ  = [697, 770, 852, 941]      # 다이얼 톤을 구성하는 하한 주파수 
HIGH_FREQ = [1209, 1336, 1477]       # 다이얼 톤을 구성하는 상한 주파수
BW = 50

lower_bounds = [0]
upper_bounds = []

for freq in LOW_FREQ:
    lower_bounds.append(freq + BW / 2)
    upper_bounds.append(freq - BW / 2)

for freq in HIGH_FREQ:
    lower_bounds.append(freq + BW / 2)
    upper_bounds.append(freq - BW / 2)

upper_bounds.append(sample_rate / 2)

print(lower_bounds)
print(upper_bounds)

In [ ]:
''' Part 7) 노이즈 제거 후 파형 출력하기 '''
filtered_amplitude = amplitude_spectrum.copy()

for i in range(0, len(lower_bounds)):
    idx = (half_frequencies >= lower_bounds[i]) & (half_frequencies < upper_bounds[i])
    filtered_amplitude[idx] = 0
    
figure(figsize=(18, 6), dpi=80)

plt.stem(half_frequencies, np.conj(filtered_amplitude), 'g', markerfmt=" ")
plt.xlabel("Frequency (Hz)");
plt.ylabel("Denoised Amplitude");
plt.title("Noise-Removed Dial Tone on Frequency Domain");
plt.grid(True);
plt.axis([0, sample_rate / 2, 0, max(abs(filtered_amplitude))]);

In [ ]:
''' Part 8) 푸리에 역변환 수행 후 파형 출력하기 '''
time_domain_amplitude_filtered = np.fft.irfft(filtered_amplitude * sample_count, sample_count)

figure(figsize=(18, 6), dpi=80)

plt.plot(time_stamps, np.conj(time_domain_amplitude_filtered))
plt.xlabel("Time (s)");
plt.ylabel("Normalized Amplitude")
plt.title("Pass-Filtered Dial Tone on Time Domain")
plt.grid(True)
plt.axis([0, (sample_count - 1) / sample_rate, min(time_domain_amplitude_filtered), max(time_domain_amplitude_filtered)])

In [39]:
''' Part 9) 노이즈가 제거된 다이얼톤 파일 들어보기 '''
denoised_audio_data = time_domain_amplitude_filtered

if audio_data.dtype == 'int16':
    scaled_data = np.int16(denoised_audio_data / np.max(np.abs(denoised_audio_data)) * 32768)
else:
    scaled_data = np.uint8(denoised_audio_data / np.max(np.abs(denoised_audio_data)) * 255)

wavfile.write(output_file, sample_rate, scaled_data)
# Audio(output_audio_file, autoplay=True)

In [ ]:
binary_amplitude = np.where((-0.6 <= time_domain_amplitude_filtered) & 
                            (time_domain_amplitude_filtered <= 0.6), 0, 1)

figure(figsize=(18, 6), dpi=80)

plt.plot(binary_amplitude)
plt.xlabel("Time (s)")
plt.ylabel("Binary Amplitude")
plt.title("Binary Dial Tone on Time Domain")
plt.show()

In [ ]:
indices = []
start_indices = []
end_indices = []

for i in range(0, len(binary_amplitude), 800):
    segment_fft = np.abs(np.fft.fft(binary_amplitude[i:i + 800]))
    if max(segment_fft) == 0:
        continue
    
    segment_indices = np.where(binary_amplitude[i:i + 800] > 0)[0]
    start_indices.append(segment_indices[0] + i)
    end_indices.append(segment_indices[-1] + i)

print("Start Indices:", start_indices)
print("End Indices:", end_indices)

In [ ]:
for i in range(len(start_indices)):
    segment_amplitude = time_domain_amplitude_filtered[start_indices[i]:end_indices[i]]

    segment_fft = np.fft.fft(segment_amplitude)
    segment_magnitude = np.abs(segment_fft)
    segment_magnitude = segment_magnitude[range(int(len(segment_magnitude) / 2))]

    for j in range(0, int(len(segment_amplitude)/2)) :
        if(segment_magnitude[j] <= 50) : segment_magnitude[j] = 0

    plt.figure(figsize=(18, 6), dpi=80)
    plt.plot(segment_magnitude)
    plt.xlim(0, len(segment_amplitude) / 2)
    plt.title(f"FFT of Segment {i + 1}")
    plt.show()

In [ ]:
low_freq_indices = []
high_freq_indices = []

for i in range(len(start_indices)):
    segment_amplitude = time_domain_amplitude_filtered[start_indices[i]:end_indices[i]]
    
    arr1 = []
    arr2 = [[]]

    segment_fft = np.fft.fft(segment_amplitude)
    segment_magnitude = np.abs(segment_fft)
    segment_magnitude = segment_magnitude[range(int(len(segment_magnitude) / 2))]

    for j in range(0, int(len(segment_amplitude)/2)) :
        if(segment_magnitude[j] <= 50) : segment_magnitude[j] = 0

    arr1 = sorted(set(segment_magnitude.tolist()), reverse=True)
    arr2[0].extend(arr1[:2])
    
    idx1 = segment_magnitude.tolist().index(arr2[0][0])
    idx2 = segment_magnitude.tolist().index(arr2[0][1])

    if idx1 > idx2 :
        arr2[0][0], arr2[0][1] = arr2[0][1], arr2[0][0]
        idx1 = segment_magnitude.tolist().index(arr2[0][0])
        idx2 = segment_magnitude.tolist().index(arr2[0][1])

    low_freq_indices.append(int(idx1 * (sample_rate / 2) / (len(segment_amplitude)/2)))
    high_freq_indices.append(int(idx2 * (sample_rate / 2) / (len(segment_amplitude)/2)))

print("Low Frequencies:", low_freq_indices)
print("High Frequencies:", high_freq_indices)

In [ ]:
low_frequencies = [697, 770, 852, 941]
high_frequencies = [1209, 1336, 1477]

dial_pad = [['1', '2', '3'], ['4', '5', '6'], ['7', '8', '9'], ['*', '0', '#']]
decoded_numbers = []

for i in range(len(low_freq_indices)):
    row, col = None, None

    for tolerance in range(10):
        if row is None:
            for idx, freq in enumerate(low_frequencies):
                if abs(low_freq_indices[i] - freq) <= tolerance:
                    row = idx
                    break
        if col is None:
            for idx, freq in enumerate(high_frequencies):
                if abs(high_freq_indices[i] - freq) <= tolerance:
                    col = idx
                    break

    if row is not None and col is not None:
        decoded_numbers.append(dial_pad[row][col])

print("Decoded Numbers:", " ".join(decoded_numbers))